In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from kerastuner.tuners import RandomSearch, BayesianOptimization, Hyperband
from kerastuner import Objective
import joblib

# Defino F1 Score metric
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred)
        self.recall.update_state(y_true, y_pred)

    def result(self):
        precision_result = self.precision.result()
        recall_result = self.recall.result()
        return 2 * (precision_result * recall_result) / (precision_result + recall_result + tf.keras.backend.epsilon())

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Cargo el archivo CSV en un DataFrame
df = pd.read_csv("datas.csv")

# Separo las características (X) y las etiquetas (y)
X = df.drop(columns=["adopted"])  # Todas las columnas excepto la columna "adopted"
y = df["adopted"]  # Columna "adopted" como etiqueta

# Divido los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Definir el keras tuner
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                                         activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()])  # Agrega las métricas

    return model

# Define the tuners
tuner_names = ['RandomSearch', 'BayesianOptimization']
tuners = [RandomSearch, BayesianOptimization]

# Perform the hyperparameter search with different tuners
best_models = {}
for tuner_name, tuner_class in zip(tuner_names, tuners):
    print(f"Performing {tuner_name} tuning:")
    tuner = tuner_class(
        build_model,
        objective=Objective("f1_score", direction="max"),
        max_trials=5,
        executions_per_trial=1,
        directory=f'my_dir_{tuner_name.lower()}',
        project_name='helloworld_6')

    tuner.search(X_train, y_train,
                 epochs=15,
                 validation_data=(X_val, y_val))

    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Build the final model with the best hyperparameters
    best_model = tuner.hypermodel.build(best_hps)

    # Train the final model
    history = best_model.fit(X_train, y_train, epochs=15, validation_data=(X_val, y_val))

    # Evaluate the final model
    evaluation = best_model.evaluate(X_test, y_test)
    print("Test Evaluation:", evaluation)

    # Get precision and F1 score from evaluation
    precision = evaluation[2]  # Assuming precision is at index 2
    f1 = evaluation[4]  # Assuming F1 score is at index 4

    # Save the best model with precision and F1 score in the filename
    model_name = f"best_model_{tuner_name.lower()}_precision_{precision:.4f}_f1_{f1:.4f}.joblib"
    joblib.dump(best_model, model_name)

    # Record the best model and evaluation
    best_models[tuner_name] = {'model': best_model, 'evaluation': evaluation}

# Write the results to a text file
with open('tuning_results.txt', 'w') as f:
    f.write("Tuning Results:\n")
    for tuner_name, result in best_models.items():
        f.write(f"\nTuner: {tuner_name}\n")
        f.write(f"Best Evaluation: {result['evaluation']}\n")
        f.write(f"Best Model: {result['model']}\n")
        f.write("\n")

Trial 5 Complete [00h 01m 54s]
f1_score: 0.8338509202003479

Best f1_score So Far: 0.8362917900085449
Total elapsed time: 00h 16m 17s
Epoch 1/15
5106/5121 [============================>.] - ETA: 0s - loss: 0.5376 - binary_accuracy: 0.7391 - precision_2: 0.7529 - recall_2: 0.9267 - f1_score: 0.8308

c:\Users\Franco-SIM\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:2620: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


5121/5121 [==============================] - 11s 2ms/step - loss: 0.5375 - binary_accuracy: 0.7392 - precision_2: 0.7530 - recall_2: 0.9268 - f1_score: 0.8309 - val_loss: 0.5288 - val_binary_accuracy: 0.7395 - val_precision_2: 0.7364 - val_recall_2: 0.9702 - val_f1_score: 0.8373
Epoch 2/15
5121/5121 [==============================] - 10s 2ms/step - loss: 0.5281 - binary_accuracy: 0.7431 - precision_2: 0.7556 - recall_2: 0.9288 - f1_score: 0.8333 - val_loss: 0.5274 - val_binary_accuracy: 0.7410 - val_precision_2: 0.7442 - val_recall_2: 0.9527 - val_f1_score: 0.8356
Epoch 3/15
5121/5121 [==============================] - 9s 2ms/step - loss: 0.5254 - binary_accuracy: 0.7450 - precision_2: 0.7569 - recall_2: 0.9296 - f1_score: 0.8344 - val_loss: 0.5248 - val_binary_accuracy: 0.7448 - val_precision_2: 0.7568 - val_recall_2: 0.9292 - val_f1_score: 0.8342
Epoch 4/15
5121/5121 [==============================] - 9s 2ms/step - loss: 0.5236 - binary_accuracy: 0.7469 - precision_2: 0.7583 - recall

c:\Users\Franco-SIM\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:2620: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


641/641 [==============================] - 1s 1ms/step - loss: 0.5203 - binary_accuracy: 0.7516 - precision_2: 0.7765 - recall_2: 0.9009 - f1_score: 0.8341
Test Evaluation: [0.5202629566192627, 0.7515988945960999, 0.7764756083488464, 0.900873601436615, 0.8340616822242737]
